<a href="https://colab.research.google.com/github/barassah/ml_experiments/blob/main/Traditional_Models_PsudolabellingDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

print("Loading libraries Done!")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Amazon_reviews.csv to Amazon_reviews.csv


In [ ]:
#reading and describing the dataset
dataset=pd.read_csv("Amazon_reviews.csv", encoding='ISO-8859-1')
dataset.head()

,Unnamed: 0,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,0,1,1,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,1,2,1,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,2,3,1,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,3,4,1,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,4,5,1,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         21000 non-null  int64 
 1   DOC_ID             21000 non-null  int64 
 2   LABEL              21000 non-null  int64 
 3   RATING             21000 non-null  int64 
 4   VERIFIED_PURCHASE  21000 non-null  object
 5   PRODUCT_CATEGORY   21000 non-null  object
 6   PRODUCT_ID         21000 non-null  object
 7   PRODUCT_TITLE      21000 non-null  object
 8   REVIEW_TITLE       21000 non-null  object
 9   REVIEW_TEXT        21000 non-null  object
dtypes: int64(4), object(6)
memory usage: 1.6+ MB


In [ ]:
dataset.describe()

,Unnamed: 0,DOC_ID,LABEL,RATING
count,21000.000000,21000.000000,21000.000000,21000.000000
mean,10499.500000,10500.500000,0.500000,4.127952
std,6062.322162,6062.322162,0.500012,1.278333
min,0.000000,1.000000,0.000000,1.000000
25%,5249.750000,5250.750000,0.000000,4.000000
50%,10499.500000,10500.500000,0.500000,5.000000
75%,15749.250000,15750.250000,1.000000,5.000000
max,20999.000000,21000.000000,1.000000,5.000000


In [ ]:
y=dataset['LABEL']
y.unique()

array([1, 0])

In [ ]:
# Text cleaning function
def clean_text(text):
    text = text.lower()
    # Modified regex to preserve words like "can't" and "don't"
    text = re.sub(r'[^\w\s]+', ' ', text) # remove punctuation but keep letters, numbers, and spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

# Apply text cleaning
dataset['REVIEW_TEXT'] = dataset['REVIEW_TEXT'].apply(clean_text)
dataset['REVIEW_TEXT']
print("Cleaning Done!!")

Cleaning Done!!


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Airline-Sentiment-2-w-AA.csv to Airline-Sentiment-2-w-AA (1).csv


In [ ]:
#reading and describing the dataset
df=pd.read_csv("Airline-Sentiment-2-w-AA.csv", encoding='ISO-8859-1')
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)


In [ ]:
df = df[['text']]  # Keep only text column
print(df)

                                                    text
0                    @VirginAmerica What @dhepburn said.
1      @VirginAmerica plus you've added commercials t...
2      @VirginAmerica I didn't today... Must mean I n...
3      @VirginAmerica it's really aggressive to blast...
4      @VirginAmerica and it's a really big bad thing...
...                                                  ...
14635  @AmericanAir thank you we got on a different f...
14636  @AmericanAir leaving over 20 minutes Late Flig...
14637  @AmericanAir Please bring American Airlines to...
14638  @AmericanAir you have my money, you change my ...
14639  @AmericanAir we have 8 ppl so we need 2 know h...

[14640 rows x 1 columns]


In [ ]:
# Text cleaning function
def clean_text(text):
    text = text.lower()
    # Modified regex to preserve words like "can't" and "don't"
    text = re.sub(r'[^\w\s]+', ' ', text) # remove punctuation but keep letters, numbers, and spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

# Apply text cleaning
df['text'] = df['text'].apply(clean_text)
print("Cleaning Done!!")

Cleaning Done!!


In [ ]:
df

,text
0,virginamerica dhepburn said
1,virginamerica plus added commercials experienc...
2,virginamerica today must mean need take anothe...
3,virginamerica really aggressive blast obnoxiou...
4,virginamerica really big bad thing
...,...
14635,americanair thank got different flight chicago
14636,americanair leaving 20 minutes late flight war...
14637,americanair please bring american airlines bla...
14638,americanair money change flight answer phones ...


In [ ]:
# Convert text data to numerical features using TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_labeled = vectorizer.fit_transform(dataset['REVIEW_TEXT'])
y_labeled = np.array(y)
X_unlabeled = vectorizer.transform(df['text'])
print("Vectorization Done!")

Vectorization Done!


In [ ]:
X_labeled

<21000x34985 sparse matrix of type '<class 'numpy.float64'>'
	with 615431 stored elements in Compressed Sparse Row format>

In [ ]:
y_labeled

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
X_unlabeled

<14640x34985 sparse matrix of type '<class 'numpy.float64'>'
	with 118329 stored elements in Compressed Sparse Row format>

In [ ]:
# Split a validation set from labeled data
X_train, X_val, y_train, y_val = train_test_split(X_labeled, y_labeled, test_size=0.3, random_state=42)



In [ ]:
# Train initial model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate on validation set
initial_acc = accuracy_score(y_val, model.predict(X_val))
print(f"Initial Model Accuracy: {initial_acc:.4f}")

Initial Model Accuracy: 0.6500


In [ ]:
# Generate pseudo-labels for unlabeled data
pseudo_labels = model.predict(X_unlabeled)

In [ ]:
# Filter high-confidence pseudo-labels
probabilities = model.predict_proba(X_unlabeled)
confidence_threshold = 0.9  # Adjust as needed
high_confidence_mask = np.max(probabilities, axis=1) > confidence_threshold
X_pseudo = X_unlabeled[high_confidence_mask]
y_pseudo = pseudo_labels[high_confidence_mask]

In [ ]:
# Combine labeled data with pseudo-labeled data
X_combined = np.vstack((X_train.toarray(), X_pseudo.toarray()))
y_combined = np.hstack((y_train, y_pseudo))

In [ ]:
# Retrain the model with combined data
model.fit(X_combined, y_combined)

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate the improved model
final_acc = accuracy_score(y_val, model.predict(X_val))
print(f"Final Model Accuracy After Pseudo-Labeling: {final_acc:.4f}")

Final Model Accuracy After Pseudo-Labeling: 0.6541


In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_val)
print("SVM Accuracy:", accuracy_score(y_val, svm_preds))


SVM Accuracy: 0.6425396825396825


In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_preds = nb.predict(X_test)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, nb_preds))


Naïve Bayes Accuracy: 0.6471428571428571


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))


Random Forest Accuracy: 0.6561904761904762


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))


Logistic Regression Accuracy: 0.6633333333333333


In [ ]:
# Print classification reports
print("SVM:\n", classification_report(y_test, svm_preds))
print("Naïve Bayes:\n", classification_report(y_test, nb_preds))
print("Random Forest:\n", classification_report(y_test, rf_preds))
print("Logistic Regression:\n", classification_report(y_test, lr_preds))


SVM:
               precision    recall  f1-score   support

           0       0.64      0.68      0.66      3111
           1       0.67      0.64      0.65      3189

    accuracy                           0.66      6300
   macro avg       0.66      0.66      0.66      6300
weighted avg       0.66      0.66      0.66      6300

Naïve Bayes:
               precision    recall  f1-score   support

           0       0.65      0.63      0.64      3111
           1       0.65      0.66      0.66      3189

    accuracy                           0.65      6300
   macro avg       0.65      0.65      0.65      6300
weighted avg       0.65      0.65      0.65      6300

Random Forest:
               precision    recall  f1-score   support

           0       0.65      0.67      0.66      3111
           1       0.66      0.65      0.66      3189

    accuracy                           0.66      6300
   macro avg       0.66      0.66      0.66      6300
weighted avg       0.66      0.66     

In [ ]:
from sklearn.metrics import precision_recall_curve

precision_curve, recall_curve, _ = precision_recall_curve(true_labels, all_probs)

plt.plot(recall_curve, precision_curve, marker=".")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()